# Лабораторная работа №6

Булгаков А.Е. 20.05.2025

In [1]:
from IPython.display import display, HTML
display(HTML("<style>" + open("customs.css").read() + "</style>"))

%config IPCompleter.greedy = True

In [2]:
# Пример ситуации, в которой будет выведена ошибка RecursionError;
def recursive():
    recursive()
recursive()

RecursionError: maximum recursion depth exceeded

Функции `getrecursionlimit` и `setrecursionlimit` из модуля **sys** позволяют получить глубину рекурсии и изменить ее:

In [3]:
import sys
sys.getrecursionlimit()

3000

**Пример рекурсивной функции:**

In [4]:
# Пример рекурсивной функции используя синтаксис def;
def dRecSum(number):
    """ Сумма числа 0 до n >= 0 """
    match number:
        case 0: return number
        case _: return number + dRecSum(number - 1)
dRecSum(0), dRecSum(2), dRecSum(10)

(0, 3, 55)

In [5]:
# Пример рекурсивной функции используя синтаксис lambda-функции;
lRecSum = lambda number: number + lRecSum(number - 1) if number else number
lRecSum(0), lRecSum(2), lRecSum(10)

(0, 3, 55)

### Постановка задачи

Напишите пользовательскую функцию **flattenIt(nestedObj, gen)** выравнивания списка на основе *рекурсии*, которая для списка **nestedObj**, содержащего различные итерируемые объекты (строка, список, кортеж, множество, словарь), возвращает либо список из всех элементов итерируемых объектов без сохранения структуры итерируемых объектов, если **gen = False**, либо функция возвращает генераторный объект, если **gen = True** , который генерирует по запросу элементы итерируемых объектов при итерационном проходе по исходному списку.

## Задание №1. Выравнивание вложенных циклов

### Подзадание №1.1. Рекурсивная функция flattenListV1

Предположим, что нам нужно выровнять следующий вложенный список:

In [6]:
itList = [[1],[2,[3]],[[[[[4]]]]],5,6,7]

Напишем рекурсивную функцию `flattenListV1(nestedObj)`, которая выравнивает вложенный список `nestedObj` и возвращает список, содержащий только элементы исходного списка без сохранения его структуры.

Для решения данной задачи будем использовать встроенную функцию **isinstance(item, type)** и синтаксис `+=`.

In [7]:
?isinstance

Signature: isinstance(obj, class_or_tuple, /)
Docstring:
Return whether an object is an instance of a class or of a subclass thereof.

A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
or ...`` etc.
Type:      builtin_function_or_method

**Реализация:**

In [8]:
def flattenListV1(nestedObj: list) -> list:
    # Создаем список в который будет добавлять элементы 
    # исходного списка, без сохранения структуры;
    result = []

    # Проверяем объект: является ли он списком ?;
    if isinstance(nestedObj, list):
        # Если да, то проходим по каждому элементу списка;
        for item in nestedObj:
            # Рекурсивно обрабатываем каждый item списка и добавляем в result;
            result += flattenListV1(item)
    else:
        # В ином случае добавляем в result, nestedObj;
        result += [nestedObj]

    # Возвращаем список с результатом выполнения функции;
    return result

flattenListV1(itList)

[1, 2, 3, 4, 5, 6, 7]

### Подзадание №1.2. Генераторная функция на основе рекурсии flattenListV2


На основе функции **flattenListV1(nestedObj)** генераторную функцию **flattenListV2(nestedObj)**, которая выравнивает вложенный список и возвращает генератнорый объект для элементов вложенного списка.

Предложим два варианта реализации: только с использованием **yield *(вариант 1)*** и с использованием **yield from *(вариант 2)***.

In [9]:
# Вариант №1 выполнения задания 1.2;
def flattenListV2(nestedObj: list):
    # Проверяем, является nestedObj списком;
    if isinstance(nestedObj, list):
        # Итерируемся по каждому элементу списка nestedObj;
        for iItem in nestedObj:
            # Рекурсивно обрабатываем каждый элемент;
            for jItem in flattenListV2(iItem):
                yield jItem
    else:
        # Если элемент не список, возвращаем его;
        yield nestedObj

[elem for elem in flattenListV2(itList)]

[1, 2, 3, 4, 5, 6, 7]

In [10]:
# Вариант №2 выполнения задания 1.2;
def flattenListV2(nestedObj: list):
    # Проверяем, является nestedObj списком;
    if isinstance(nestedObj, list):
        # Итерируемся по каждому элементу списка nestedObj;
        for item in nestedObj:
            # Рекурсивно раскрываем вложенные структуры;
            yield from flattenListV2(item)
    else:
        # Возвращаем конечный элемент;
        yield nestedObj

[elem for elem in flattenListV2(itList)]

[1, 2, 3, 4, 5, 6, 7]

### Подзадание №1.3. Функция flattenList: создание, документирование, тестирование

Объедините две пользовательские функции `flattenListV1` и `flattenListV2` в одну пользовательскую функцию `flattenList(nestedObj, gen)`, которая возвращает либо список из элементов вложенных списков без сохранения структуры, если `gen = False`, либо возвращает генераторный объект, если `gen = True`. Стандартным значением для аргумента **gen** является объект **False**.


In [11]:
def flattenList(nestedObj: list, gen: bool = False):
    """
    Рекурсивно преобразует вложенный список в одномерную структуру.
    
    Возвращает все элементы исходного списка в виде одномерного списка или генератора,
    сохраняя порядок элементов. Обрабатывает списки любой глубины вложения.

    Параметры:
        nestedObj: list
            Вложенный список произвольной глубины для преобразования.
        gen: bool, optional (по умолчанию False)
            Определяет тип возвращаемого значения (указано в пункте "Возвращает").

    Возвращает:
        list или generator:
            1) Если gen = False: плоский список всех элементов;
            2) Если gen = True: генератор, последовательно возвращающий элементы.
    """


    # Фукнкция возвращающая одномерный список;
    def flattenListV1(nestedObj: list) -> list:
        result = []
        if isinstance(nestedObj, list):
            for item in nestedObj:
                result += flattenListV1(item)
        else:
            result += [nestedObj]
            
        return result


    # Генераторная функция;
    def flattenListV2(nestedObj: list):
        if isinstance(nestedObj, list):
            for item in nestedObj:
                yield from flattenListV2(item)
        else:
            yield nestedObj

    # Словарь, с парами (значение gen, вызываемая функция);
    genDict = {"True": flattenListV2,
              "False": flattenListV1}

    return genDict[str(gen)](nestedObj)

In [12]:
# Выведем строки документации;
print(flattenList.__doc__, end = "")


Рекурсивно преобразует вложенный список в одномерную структуру.

Возвращает все элементы исходного списка в виде одномерного списка или генератора,
сохраняя порядок элементов. Обрабатывает списки любой глубины вложения.

Параметры:
    nestedObj: list
        Вложенный список произвольной глубины для преобразования.
    gen: bool, optional (по умолчанию False)
        Определяет тип возвращаемого значения (указано в пункте "Возвращает").

Возвращает:
    list или generator:
        1) Если gen = False: плоский список всех элементов;
        2) Если gen = True: генератор, последовательно возвращающий элементы.


**Примеры вызова функции flattenList:**

In [13]:
# Пример №1; 
exList = [[[1]], 2, [5], [[[[3]]]], 1]
print(f"Пример вызова функции flattenList для {exList}, если gen не указан {flattenList(exList)}\n")

# Пример №2;
print(f"Пример вызова функции flattenList для {exList}, если gen = False не указан {flattenList(exList, gen = False)}\n")

# Пример №3;
print(f"Пример вызова функции flattenList для {exList}, если gen = True {flattenList(exList, True)}")

Пример вызова функции flattenList для [[[1]], 2, [5], [[[[3]]]], 1], если gen не указан [1, 2, 5, 3, 1]

Пример вызова функции flattenList для [[[1]], 2, [5], [[[[3]]]], 1], если gen = False не указан [1, 2, 5, 3, 1]

Пример вызова функции flattenList для [[[1]], 2, [5], [[[[3]]]], 1], если gen = True <generator object flattenList.<locals>.flattenListV2 at 0x00000196D11166C0>


Как мы видим, 3-ий вывод, при `gen = True` будет возвращаться генераторный объект `<generator object flattenList.<locals>.flattenListV2 at 0x00000264169CA9B0>`. Чтобы вывести его, воспользуемся следующей структурой:

In [14]:
[elem for elem in flattenList(exList, True)]

[1, 2, 5, 3, 1]

## Задание №2. Выравнивание вложенных итерируемых объектов произвольного типа


**Создадим различные итерируемые объекты**: список, кортеж, строка, словарь, генераторный объект, файловый объект

In [40]:
# Итерируемый объект список;
itList 

# Итерируемый объект кортеж;
itTuple = (((7), (8)), (9), 10)

# Итерируемый объект строка;
itStr = "abcdefgh"

# Итерируемый объект словарь;
itDict = {"key1": 11, "key2": {"key3": 12}}

# Генератореый объект;
itGen = ([step, step ** 2, step ** 3] for step in range(5))

# Файловый объект;
itFile = open("flattenFile.txt")

Объединим все итерируемые объекты в список:

In [41]:
it = [itList, itTuple, itStr, itDict, itGen, itFile]

Все итерируемые объекты имеют метод `__iter__` для создания объекта итератора:

In [17]:
# Это генератор списка (list comprehension), который проверяет, 
# является ли каждый элемент итерируемого объекта it итератором или итерируемым.
['__iter__' in dir(_) for _ in it]

[True, True, True, True, True, True]

Функция `hasattr()` проверяет, существует ли указанный атрибут или метод у объекта:

In [18]:
[hasattr(_, '__iter__') for _ in it]

[True, True, True, True, True, True]

In [19]:
?hasattr

Signature: hasattr(obj, name, /)
Docstring:
Return whether the object has an attribute with the given name.

This is done by calling getattr(obj, name) and catching AttributeError.
Type:      builtin_function_or_method

### Подзадание №2.1. Рекурсивная функция flattenItV1

На основе функции `flattenListV1(nestedObj)` напишем пользовательскую функцию `flattenItV1(nested_obj)` одного аргумента, которая выравнивает вложенную структуру из различных итерируемых объектов и возвращает список, содержащий только элементы без сохранения структуры.

Для этого вместо проверки `isinstance(nestedObj, list)` будем использовать проверку на итерируемость объекта `hasattr(nestedObj, '__iter__')`.

При этом дополнительно учтем, что если итерируемым объектом является:
1) Строка, то ее мы не итерируем;
2) Словарь, то для его итерирования нужно выполнить дополнительные преобразования, чтобы осущесвить проход как по ключам, так и по значениям.

In [20]:
def flattenItV1(nestedObj):
    result = []
    # Проверяем объект nestedObj: он должен быть итерируемым
    # и не иметь тип str (этого требует условие);
    if hasattr(nestedObj, "__iter__") and not isinstance(nestedObj, str):
        # Обработка словаря требует отдельного синтаксиса, поэтому проверяем, 
        # является ли nestedObj им;
        if isinstance(nestedObj, dict):
            # Отдельно проходимся по каждой паре (key, item), предварительно
            # полученных используя метод dict.items();
            for dictItem in nestedObj.items():
                result += flattenItV1(dictItem) # Рекурсия;
        else:
            # Отедльный цикл обработки других итерируемых объектов;
            for item in nestedObj:
                result += flattenItV1(item)
    else:
        # В противном случае nestedObj не итерируем и добавляем его result;
        result += [nestedObj]

    # Возвращаем список result;
    return result

# Воспользуемся функцией flattenListV1 для ранее созданного списка it;
flattenItV1(it)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 7,
 8,
 9,
 10,
 'abcdefgh',
 'key1',
 11,
 'key2',
 'key3',
 12,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 4,
 8,
 3,
 9,
 27,
 4,
 16,
 64,
 'Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore \n',
 'veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia \n',
 'consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, \n',
 'adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis \n',
 'nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea \n',
 'voluptate velit esse quam nihil molestiae consequatur, v

### Подзадание №2.2. Генераторная функция на основе рекурсии flattenItV2


На основе функций `flattenItV1` и `flattenListV2` напишем пользовательскую функцию `flattenItB2(nestedObj)`, которая **выравнивает вложенную структуру из различных итерируемых объектов** и возвращает генератнорый объект элементов без сохранения структуры.

In [23]:
# Генераторная функция;
def flattenItV2(nestedObj):
    result = []
    # Проверяем объект nestedObj: он должен быть итерируемым
    # и не иметь тип str (этого требует условие);
    if hasattr(nestedObj, "__iter__") and not isinstance(nestedObj, str):
        # Обработка словаря требует отдельного синтаксиса, поэтому проверяем, 
        # является ли nestedObj им;
        if isinstance(nestedObj, dict):
            # Отдельно проходимся по каждой паре (key, item), предварительно
            # полученных используя метод dict.items();
            for dictItem in nestedObj.items():
                yield from flattenItV2(dictItem) # Рекурсия с использованием синтаксиса yield;
        else:
            # Отдельный цикл обработки других итерируемых объектов;
            for item in nestedObj:
                yield from flattenItV2(item) # Рекурсия с использованием синтаксиса yield;
    else:
        yield nestedObj

# Воспользуемся функцией flattenListV2 для ранее созданного списка it;
[elem for elem in flattenItV2(it)]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 7,
 8,
 9,
 10,
 'abcdefgh',
 'key1',
 11,
 'key2',
 'key3',
 12,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 4,
 8,
 3,
 9,
 27,
 4,
 16,
 64,
 'Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore \n',
 'veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia \n',
 'consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, \n',
 'adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis \n',
 'nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea \n',
 'voluptate velit esse quam nihil molestiae consequatur, v

### Подзадание №2.3. Функция flattenIt: создание, документирование, тестирование

Объединим две пользовательские функции `flattenItV1` и `flattenItV2` в одну функцию `flattenIt(nestedObj, gen)`, которая возвращает либо список из элементов вложенных объектов без сохранения структуры, если **gen = False**, либо возвращает генераторный объект, если **gen = True**. Стандартным значением для аргумента **gen** является объект **False**.

In [24]:
def flattenIt(nestedObj, gen: bool = False):

    """
    Рекурсивно преобразует вложенные итерируемые объекты в одномерную структуру.
    
    Поддерживает обработку списков, словарей, кортежей и других iterable-объектов,
    исключая строки (они обрабатываются как атомарные элементы).

    Параметры:
        nestedObj: Any
            Вложенная структура данных произвольной сложности.
            Может содержать комбинации списков, словарей и других итерируемых объектов.
        gen: bool, optional (default=False)
            Флаг определения формата вывода (подробнее в пункте "Возвращает")

    Возвращает:
        list или generator:
            1) При gen = False: одномерный список всех элементов;
            2) При gen = True: генератор элементов.
    """


    # Фукнкция возвращающая одномерный список;
    def flattenItV1(nestedObj):
        result = []
        if hasattr(nestedObj, "__iter__") and not isinstance(nestedObj, str):
            if isinstance(nestedObj, dict):
                for dictItem in nestedObj.items():
                    result += flattenItV1(dictItem)
            else:
                for item in nestedObj:
                    result += flattenItV1(item)
        else:
            result += [nestedObj]
    
        return result


    # Генераторная функция;
    def flattenItV2(nestedObj):
        result = []
        if hasattr(nestedObj, "__iter__") and not isinstance(nestedObj, str):
            if isinstance(nestedObj, dict):
                for dictItem in nestedObj.items():
                    yield from flattenItV2(dictItem)
            else:
                for item in nestedObj:
                    yield from flattenItV2(item)
        else:
            yield nestedObj


    # Словарь, с парами (значение gen, вызываемая функция);
    genDict = {"True": flattenItV2,
              "False": flattenItV1
              }

    return genDict[str(gen)](nestedObj)

In [25]:
# Выведем строки документации;
print(flattenIt.__doc__, end = "")


Рекурсивно преобразует вложенные итерируемые объекты в одномерную структуру.

Поддерживает обработку списков, словарей, кортежей и других iterable-объектов,
исключая строки (они обрабатываются как атомарные элементы).

Параметры:
    nestedObj: Any
        Вложенная структура данных произвольной сложности.
        Может содержать комбинации списков, словарей и других итерируемых объектов.
    gen: bool, optional (default=False)
        Флаг определения формата вывода (подробнее в пункте "Возвращает")

Возвращает:
    list или generator:
        1) При gen = False: одномерный список всех элементов;
        2) При gen = True: генератор элементов.


**Примеры вызова функции flattenList:**

In [26]:
# Пример №1; 
ex1 = [[[1]], 2, [5], (1, ("apple")), [[[[3]]]], 1, "affdgd"]
print(f"Пример вызова функции flattenList для {exList}, если gen не указан {flattenIt(ex1)}\n")

# Пример №2;
ex2 = {"apple": {"banana": (1, 2)}, "key1": [3, 4]}
print(f"Пример вызова функции flattenList для {exList}, если gen = False не указан {flattenIt(ex2, gen = False)}\n")

# Пример №3;
ex3 = [lRecSum(number) for number in range(10)]
print(f"Пример вызова функции flattenList для {exList}, если gen = True: {flattenIt(ex3, True)} — " +
      f"как мы видим функция вернула генераторный объект. Выведем его элементы: {[_ for _ in flattenIt(ex3, True)]}")

Пример вызова функции flattenList для [[[1]], 2, [5], [[[[3]]]], 1], если gen не указан [1, 2, 5, 1, 'apple', 3, 1, 'affdgd']

Пример вызова функции flattenList для [[[1]], 2, [5], [[[[3]]]], 1], если gen = False не указан ['apple', 'banana', 1, 2, 'key1', 3, 4]

Пример вызова функции flattenList для [[[1]], 2, [5], [[[[3]]]], 1], если gen = True: <generator object flattenIt.<locals>.flattenItV2 at 0x00000196D16DB880> — как мы видим функция вернула генераторный объект. Выведем его элементы: [0, 1, 3, 6, 10, 15, 21, 28, 36, 45]


## Задание №3. Обработка циклических объектов

**Циклическими объектами в Python** могут быть списки и словари. Циклический список содержит элемент, который ссылается на весь список. Циклический словарь содержит элемент `<ключ>:<значение>`, `<значение>` которого сылается на весь словарь.

Создадим циклический объект списка:

In [27]:
itListCyclic = itList[:]
itListCyclic.append(itListCyclic)

Для отображения циклических объектов в Python используется специальный синтаксис:

In [28]:
itListCyclic

[[1], [2, [3]], [[[[[4]]]]], 5, 6, 7, [...]]

Обработаем ситуацию выравнивания вложенного списка из **Задания 1.1**, когда вложенный список является циклическим объектом.

Для этого изменим код пользовательской функции `flattenListV1` из **Задания 1.1**, добавив генерацию исключения `ValueError` с помощью оператора **raise**, когда текущий элемент **item** списка **nestedObj** ссылается на этот же список

In [29]:
def flattenListV1(nestedObj):
    result = []
    if isinstance(nestedObj, list):
        for item in nestedObj:
            if item is nestedObj:
                raise ValueError("Cyclic list is found") # Генерация исключения;
            else:
                result += flattenListV1(item)
    else:
        result += [nestedObj]
    
    return result

Протестируем функцию `flattenListV1`:

In [30]:
# Не на циклическом объекте;
flattenListV1(itList)

[1, 2, 3, 4, 5, 6, 7]

In [32]:
# На циклическом объекте;
flattenListV1(itListCyclic)

ValueError: Cyclic list is found

Для того, чтобы стандартное сообщение об ошибке не выводилось, перехватим созданное нами исключение `ValueError` внутри функции `flattenListV1` с использованием оператора **try** в сочетании с **except**. Если внутри блока **try** возникнет исключение `ValueError`, то процесс выполнения кода перейдет в блок оператора **except**, для которого указано исключение ValueError:

In [33]:
try:
 print(flattenListV1(itListCyclic))
except ValueError as text:
 print(text)

Cyclic list is found


### Подзадание №3.1. Доопределение функции flattenIt

Доопределим пользовательскую функцию `flattenIt` из **Задания 6.2**, добавив генерацию исключения `ValueError` с помощью оператора raise при обработке циклических списков и циклических словарей.


In [36]:
def flattenIt(nestedObj, gen: bool = False):

    """
    Рекурсивно преобразует вложенные итерируемые объекты в одномерную структуру.
    
    Поддерживает обработку списков, словарей, кортежей и других iterable-объектов,
    исключая строки (они обрабатываются как атомарные элементы).

    Параметры:
        nestedObj: Any
            Вложенная структура данных произвольной сложности.
            Может содержать комбинации списков, словарей и других итерируемых объектов.
        gen: bool, optional (default=False)
            Флаг определения формата вывода (подробнее в пункте "Возвращает")

    Возвращает:
        list или generator:
            1) При gen = False: одномерный список всех элементов;
            2) При gen = True: генератор элементов.

    Исключения:
        ValueError: 
            Возникает при обнаружении циклических ссылок во входной структуре
    """

    
    
    def flattenItV1(nestedObj, firstLevel = None):
        if firstLevel is None:
            firstLevel = nestedObj
        result = []
        
        if hasattr(nestedObj, "__iter__") and not isinstance(nestedObj, str):
            if isinstance(nestedObj, dict):
                for key, value in nestedObj.items():
                    if value is firstLevel: 
                        raise ValueError("Cyclic object is found")
                    result += flattenItV1((key, value), firstLevel)
            else:
                for item in nestedObj:
                    if item is firstLevel:
                        raise ValueError("Cyclic object is found")
                    result += flattenItV1(item, firstLevel)
        else:
            result += [nestedObj]
        
        return result


    
    
    def flattenItV2(nestedObj):
        if hasattr(nestedObj, "__iter__") and not isinstance(nestedObj, str):
            if isinstance(nestedObj, dict):
                for dictItem in nestedObj.items():
                    yield from flattenItV2(dictItem)
            else:
                for item in nestedObj:
                    if item is nestedObj:
                        raise ValueError("Cyclic object is found")
                    yield from flattenItV2(item)
        else:
            yield nestedObj

    
    
    genDict = {"True": flattenItV2,
              "False": flattenItV1
              }

    return genDict[str(gen)](nestedObj)

### Подзадание №3.2. Тестирование функции flattenIt

Протестируем функцию **flattenIt**, перехватывая исключение `ValueError` с помощью операторов **try/except** для циклических объектов различных типов.


In [44]:
# Тестируем для itListCyclic;
itListCyclic

try:
    print(flattenIt(itListCyclic))
except ValueError as text:
    print(text)

Cyclic object is found


**Пояснение ошибки**

Функция `flattenIt` ловит прямой цикл **(itListCyclic)**, потому что там список явно содержит сам себя, и проверка **if item is nestedObj** срабатывает. Но она не ловит вложенный цикл **(testCyclic)**, потому что там словарь ссылается на себя через другой словарь, а проверка сравнивает только с непосредственным родителем, а не со всеми предыдущими объектами. Для починки нужно хранить `id` всех обработанных объектов и проверять, не встречался ли текущий объект раньше в цепочке рекурсии.

In [45]:
# Тестируем для testCyclic;
testCyclic = {'a' : 1}
testCyclic['b'] = {2 : testCyclic}

try:
    print(flattenIt(testCyclic))
except ValueError as text:
    print(text)

Cyclic object is found


**Протестируем обычные случаи:**

In [39]:
# Для itTuple;
try:
    print(flattenIt(itTuple))
except ValueError as text:
    print(text)

[7, 8, 9, 10]


In [42]:
# Для it;
try:
    print(flattenIt(it))
except ValueError as text:
    print(text)

[1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 'abcdefgh', 'key1', 11, 'key2', 'key3', 12, 0, 0, 0, 1, 1, 1, 2, 4, 8, 3, 9, 27, 4, 16, 64, 'Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore \n', 'veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia \n', 'consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, \n', 'adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis \n', 'nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea \n', 'voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo v

In [43]:
# Для itList;
try:
    print(flattenIt(itList))
except ValueError as text:
    print(text)

[1, 2, 3, 4, 5, 6, 7]
